# HPAT
High Performance Analytics Toolkit (HPAT) scales analytics/ML codes in Python to bare-metal cluster/cloud performance automatically. It compiles a subset of Python (Pandas/Numpy) to efficient parallel binaries with MPI, requiring only minimal code changes.

## Linear Regression
Linear regression is a linear approach to modelling the relationship between a scalar response (or dependent variable) and one or more explanatory variables (or independent variables).

This exercise demostrates how HPAT can easily distribute a hand-written algorithm for linear regression fitting/prediction. HPAT automatically distributes file-IO and computation.

https://github.com/IntelLabs/hpat

## Create an input file (hdf5)

In [ ]:
# boilerplate code
import h5py
import numpy as np
import argparse
import time
import hpat

N = 100000
D = 16
p = 32
file_name = 'lir.hdf5'

In [ ]:
def gen_lir(N, D, p, file_name):
    # np.random.seed(0)
    points = np.random.random((N,D))
    responses = np.random.random((N,p))
    f = h5py.File(file_name, "w")
    dset1 = f.create_dataset("points", (N,D), dtype='f8')
    dset1[:] = points
    dset2 = f.create_dataset("responses", (N,p), dtype='f8')
    dset2[:] = responses
    f.close()

gen_lir(N, D, p, file_name)

In [ ]:
!ls -l *.hdf5

## Read data from file into pandas and perform linear regression training/fitting

In [ ]:
def linear_regression(iterations):
    f = h5py.File("lir.hdf5", "r")
    X = f['points'][:]
    Y = f['responses'][:]
    f.close()
    N,D = X.shape
    p = Y.shape[1]
    alphaN = 0.01/N
    w = np.zeros((D,p))
    t1 = time.time()
    for i in range(iterations):
        w -= alphaN * np.dot(X.T, np.dot(X,w)-Y)
    t2 = time.time()
    print("Execution time:", t2-t1)
    return w

In [ ]:
linear_regression(100)

TODO 1. Enable HPAT by decorating linear_regressino with hpat.jit

TODO 2. Run on multiple processes with MPI (after saving right cells to a python file)

In [ ]:
%save -f runme.py 1 6 7

In [ ]:
!mpirun -n 2 python runme.py